## Extracting Deep Cognitive Features from Bridge Inspectors Eye Tracking Data

Previous to this study we collected data from five different PhD engineering students on the task of inspecting a bridge in a virtual environment. The students were given 3 minutes to inspect a bridge inside of a Unity platform that we have developed, and take images of any defects, cracks, or areas of concern that they found on the bridge. During their inspection we collected data of their character's movement as well as eye movement. Below you see an example of the type of data that was obtained at one time step:

```xml
<?xml version="1.0" encoding="utf-8"?>
<Data>
  <GazeData>
    <DisplayDimensions Width="1920" Height="1080" />
    <Timestamp>902049965729</Timestamp>
    <GazeOrigin>
      <CombinedGazeRayScreen Origin="(7656.03600000, 927.89240000, -1049.60500000)" Direction="(-0.95211820, -0.28659020, 0.10647600)" Valid="True" />
    </GazeOrigin>
    <pupil average_pupildiameter="4.157799" />
    <IntersectionPoint X="4420.004" Y="-46.16226" Z="-687.7166" />
    <HitObject Name="RW_Bridge_Vologda_II_track_LOD0">
      <ObjectPosition X="0" Y="0" Z="0" />
    </HitObject>
    <PositionOnDisplayArea X="0.5182106" Y="0.5221348" />
  </GazeData>
  <CameraData>
    <Timestamp>902049965729</Timestamp>
    <CameraOrigin X="7656.323" Y="927.9785" Z="-1049.637" />
    <CameraDirection X="-0.9624248" Y="-0.2623872" Z="0.06994079" />
  </CameraData>
<Data>
```

One can see information such as the time of the data point, the location and viewing direction of the inspector at this time as well as the location where their eye were looking. This data was collected at a rate of 60 fps, therefore for 3 minutes of data we have 5400 data points.

### Goal

Our goal in this experiment is to determine whether we can use machine learning to make insights into the cognitive behavior of the inspectors based on this eye tracking data. For example, a successful insight would be if we can correctly identify whether a person is planning, searching, or deciding at every point in their search. For example, when the person is first planning where on the bridge to look, then the person might be actively looking, once the person finds something interesting such as a crack they will be deciding whether this is something they should take a picture of or not.

In addition to these fine-scaled, granular behavioral patterns, we would also like to see if machine learning methods can identify more "big picture" patterns, such as classifying search styles based on a larger set of data.

We believe that the insights gathered from these methods can be used as tools to make concrete data-driven decisions for designing training procedures for new inspectors, or comparing the efficiency of different inspection patterns.

### Methodology

In order to extract deep features from our data we propose two methods. 

The first method consists of using dimensionality reduction and clustering techniques such as PCA, t-SNE, and k-means on manually extracted feature vectors, such as velocity, acceleration, rate of fixations to saccades, etc. Then investigating these reduced dimensionality arrays to determine which combination of features creates the most meaningful feature space.

The second method involves using similar dimensionality reduction techniques on feature vectors extracted from unsupervised feature extraction techniques such as Deep Autoencoders, Variational Autoencoders, LSTM Autoencoders, etc.

We will then visualize the two methods and extract clusters of points, then we will interpret the clusters to find whether they correspond to certain parts of the inspection process or different search patterns. We can apply both of these methods on a global as well as local level to get different types of clustering.

For example, if we apply the method "locally" meaning that we divide the dataset into 1 second intervals and perform the clustering on these 1 second intervals then we expect the clusters to represent local/short term cognitive behavior such as whether a person is searching or planning.

On the other hand, if we perform the clustering "globally" meaning that we use the entire data record we expect the clusters to represent more global characteristics about the search, such as search strategy, or who the inspector is.

Something to keep in mind is that we want to anonymize the data records as much as possible to avoid clustering based on trivial factors such as the exact locations that inspectors looked at or other features that wouldn't generalize well and wouldn't truly represent an inspector's cognitive behavior.

### Table of Contents (TO-DO)


<ul>
    <li>Loading Data</li>
    <li>Local clustering</li>
        <ul>
            <li>Breaking up the data</li>
            <li>Method 1
                <ul>
                    <li>Extracting manual features</li>
                    <li>Trying different clustering techniques</li>
                    <li>Visualizing clusters</li>
                </ul>
            </li>
            <li>Method 2
                <ul>
                    <li>Training Deep AE, VAE, and LSTM AE</li>
                    <li>Extracting deep features</li>
                    <li>Clustering / dimensionality reduction of deep features</li>
                    <li>Visualizing clusters</li>
                </ul>
            </li>
            <li>Testing generality of clusters</li>
            <li>Findings and conclusions</li>
        </ul>
    <li>Global clustering
        <ul>
            <li>Extracting manual global features</li>
            <li>Extracting deep features (Training extractor model)</li>
            <li>Clustering techniques</li>
            <li>Visualizing clusters</li>
            <li>Testing generality of clusters</li>
            <li>Findings and conclusions</li>
        </ul>
    </li>
</ul>

## Loading Data <a name="loading-data"></a>

In this section we will load the data from the different users' xml files.

The data is located in the test_1 folder. The files are divided into two data records for each user, the first data record is for the task of bridge inspection (with suffix "_truss.xml"). The second data record is for the task of looking for a certain object inside a warehouse (with suffix "_warehouse.xml"). We will use the second data record for testing the generality of our clusters, but not for generating the classifiers themselves.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm # Use notebook version for better display
import importlib
import gc
import pickle
import data_loader
importlib.reload(data_loader)
import manual_features
importlib.reload(manual_features)

In [ ]:
# Set plot style
sns.set_theme(style="whitegrid", palette="muted", color_codes=True)

# Specify the folder containing the XML data
DATA_FOLDER = "test_1" # Make sure this folder exists relative to the notebook
# Load the data
truss_data, warehouse_data = data_loader.load_all_data(DATA_FOLDER)

# Display some basic info about the loaded data
if truss_data:
    # Example: Display head of the first truss dataset
    first_key = list(truss_data.keys())[0]
    print(f"\nHead of data for '{first_key}':")
    first_key_df = data_loader.data_dict_to_df(truss_data[first_key])
    display(first_key_df.head())
    print(f"\nInfo for '{first_key}':")
    first_key_df.info()
print("\nTruss Data Keys:", list(truss_data.keys()))
print("\nWarehouse Data Keys:", list(warehouse_data.keys()))

## Local Clustering
### Breaking up the data <a name="break-up-data"></a>

For performing local clustering we would like to gather features that represent cognitive behavior at one instance in time. For example, is the person planning right now or are they actively searching? To achieve this we propose to first break up our data records into small instances of time (such as 1s) and then perform the clustering and subsequent analysis on these short time records.

Equally as important, we must anonymize these short time records in order to avoid clustering based on trivial facts. For example, if not anonymized, the clustering analysis could yield clusters that simply divide the data based on location (Such as looking at the top of the deck vs the bottom) but these clusters do not provide any insight into the person's cognitive behavior.

In [ ]:
# Define constants
WINDOW_SECONDS = 0.5 # Duration of each local window
SAMPLING_RATE = 60 # Hz (data points per second)
WINDOW_POINTS = int(WINDOW_SECONDS * SAMPLING_RATE)
WINDOW_OVERLAP = 0.8 # Percentage of overlap

print(f"Window size: {WINDOW_SECONDS} seconds ({WINDOW_POINTS} data points)")

USE_SAVED = True  # Set to True to skip processing
SAVE_TO_FILE = "windows_anonymized.pkl"


if USE_SAVED:
    print("Loading saved anonymized windows...")
    with open(SAVE_TO_FILE, "rb") as f:
        all_windows_anonymized = pickle.load(f)
else:
    # Debug mode: reduce load while testing
    all_windows_anonymized = {}
    TEST_KEYS = list(truss_data.keys())[:2]  # Just 2 first for now
    for key in tqdm(TEST_KEYS, desc="Creating & Anonymizing Windows"):
        df = data_loader.data_dict_to_df(truss_data[key])
        zero_sensitive_cols = [
            'GazeOrigin_X', 'GazeOrigin_Y', 'GazeOrigin_Z',
            'GazeDirection_X', 'GazeDirection_Y', 'GazeDirection_Z'
        ]

        df = data_loader.fill_dataframe(df, zero_sensitive_cols=zero_sensitive_cols)

        ''' ######### THIS IS PROVISIONAL  #################
        # Save raw data before windowing
        raw_csv_filename = f"raw_data_{key}.csv"
        df.to_csv(raw_csv_filename, index=False)
        print(f"Raw data for '{key}' saved to {raw_csv_filename}")#'''
        
        windows_raw = data_loader.create_windows(df, WINDOW_POINTS, WINDOW_OVERLAP)
        anonymized_windows = [data_loader.anonymize_window(w) for w in windows_raw]
        all_windows_anonymized[key] = anonymized_windows
        print(f"Processed '{key}': Created {len(windows_raw)} raw windows -> {len(anonymized_windows)} anonymized windows.")
        
        # Free memory
        del df, windows_raw, anonymized_windows
        gc.collect()
    
    '''# Process all truss data
    all_windows_anonymized = {}
    for key, data in tqdm(truss_data.items(), desc="Creating & Anonymizing Windows"):
        df = data_loader.data_dict_to_df(data)
        windows_raw = data_loader.create_windows(df, WINDOW_POINTS, WINDOW_OVERLAP)
        anonymized_windows = [data_loader.anonymize_window(w) for w in windows_raw]
        all_windows_anonymized[key] = anonymized_windows
        print(f"Processed '{key}': Created {len(windows_raw)} raw windows -> {len(anonymized_windows)} anonymized windows.")'''

        
    with open(SAVE_TO_FILE, "wb") as f:
        pickle.dump(all_windows_anonymized, f)
        print(f"\nSaved anonymized windows to '{SAVE_TO_FILE}'")

# Display and Debug First Anonymized Window
if all_windows_anonymized:
    first_key = list(all_windows_anonymized.keys())[0]
    '''
    ########### THIS IS PROVSIONAL##############
    if len(all_windows_anonymized[first_key]) > 9:
        window_10_df = all_windows_anonymized[first_key][9]  # Window index 9 = 10th window
        output_filename = f"anonymized_window10_{first_key}.csv"
        window_10_df.to_csv(output_filename, index=False)
        print(f"\nWindow 10 saved to: {output_filename}")
    else:
        print(f"\nLess than 10 windows available for '{first_key}'")#'''

    if all_windows_anonymized[first_key]:
        # Inspect structure and details of anonymized window
        first_window_df = all_windows_anonymized[first_key][0]
        print(f"\nInfo for anonymized window of '{first_key}':")
        first_window_df.info()
    else:
        print(f"No windows created for '{first_key}'.")
    del first_key
    gc.collect()

# Optional Debug: Check windowing step size and start times
first_key = list(all_windows_anonymized.keys())[0]
raw_df = data_loader.data_dict_to_df(truss_data[first_key])
windows_raw = data_loader.create_windows(raw_df, WINDOW_POINTS, WINDOW_OVERLAP)

print("\nChecking start times of the first 5 windows:")
for i, w in enumerate(windows_raw[:5]):
    start_time = w['Time_sec'].iloc[0]
    end_time = w['Time_sec'].iloc[-1]
    print(f"Window {i}: Start = {start_time:.3f}, End = {end_time:.3f}, Rows = {len(w)}")

step_size = int(WINDOW_POINTS * (1 - WINDOW_OVERLAP))
print(f"\nStep size between windows: {step_size} samples")
del windows_raw, first_window_df
gc.collect();

### Method 1
### Extracting manual features <a name="manual-feature-extraction"></a>

After anonymizing the data records we will do some manual feature engineering on our data in order to provide more meaningful features for our clustering algorithms. Based on previous literature we have decided to include the following features:

1. Eye metrics
   - Average pupil diameter (mm)
   - Average eye movement velocity (deg/s)
   - Average eye movement acceleration (deg/s/s)
   - Ratio of fixation/saccade (s)
   - Number of fixations
   - Average fixation duration (s)
   - Spatial variance of fixations (deg)
   - Mean saccade velocity (deg/s)
   - Mean saccade amplitude (deg)
1. Movement patterns (within 1s window)
   - Average movement velocity (m/s)
   - Average turning velocity (deg/s)
   - Total action, including translation and rotation

These features are selected to capture cognitive behavior patterns
while being meaningful within a 1-second analysis window

In [ ]:
FEATURES_FILE = "manual_features_saved.pkl"
FEATURES_SAVED = False  # Set to False to force recomputation

'''
FEATURES_FILE = "manual_features.csv"
FEATURES_SAVED = True  # Set to False to force recomputation

# === Load if already computed ===
if FEATURES_SAVED:
    print(f"Loading features from '{FEATURES_FILE}'...")
    manual_features_df = pd.read_csv(FEATURES_FILE)'''

# === Load if already computed ===
if FEATURES_SAVED:
    print(f"Loading features from '{FEATURES_FILE}'...")
    with open(FEATURES_FILE, "rb") as f:
        manual_features_df = pickle.load(f)
else:
    print("Calculating manual features...")
    # Calculate features for all anonymized windows
    all_manual_features = []
    window_indices = [] # Keep track of which user/window each feature row corresponds to

    for key, anonymized_windows in tqdm(all_windows_anonymized.items(), desc="Calculating Manual Features"):
        for i, window_df in enumerate(anonymized_windows):
            features = manual_features.calculate_manual_features_for_window(window_df)
            all_manual_features.append(features)
            window_indices.append({'user_task': key, 'window_index': i})

    # Create DataFrames
    manual_features_df = pd.DataFrame(all_manual_features)
    window_indices_df = pd.DataFrame(window_indices)

    # Combine indices with features
    manual_features_df = pd.concat([window_indices_df, manual_features_df], axis=1)

    # Save to file for future use
    with open(FEATURES_FILE, "wb") as f:
        pickle.dump(manual_features_df, f)
    print(f"Features saved to '{FEATURES_FILE}'")

print("\nManual Features DataFrame:")
display(manual_features_df.head())
print("\nInfo:")
manual_features_df.info()
print("\nDescription:")
display(manual_features_df.describe())

### Trying different clustering techniques <a name="clustering1"></a>

In this section we will apply clustering techniques to the manual features created above. We will apply the following techniques and explore the clusters created.

1. PCA
2. T-SNE
3. K-means
4. Hierarchical clustering

In [ ]:
'''# Load the data from the .pkl file
with open('manual_features_saved.pkl', 'rb') as file:
    data = pickle.load(file)

# Create a Pandas DataFrame from the loaded data
df = pd.DataFrame(data)

# Save the DataFrame to a .csv file
df.to_csv('manual_features_saved.csv', index=False)
raise SystemExit("Stopping here on purpose.")
#'''

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

# --- Data Preparation ---
# Select only the numeric feature columns for clustering
feature_columns = manual_features_df.select_dtypes(include=np.number).columns
# Drop columns that might be constant or have zero variance if they exist
# Also drop index/identifier columns if they were included by mistake
feature_columns = feature_columns.drop(['window_index'], errors='ignore')

print(f"Using features for clustering: {feature_columns.tolist()}")

# Handle potential NaN/Inf values if not already done during feature extraction
features_for_clustering = manual_features_df[feature_columns].fillna(0).replace([np.inf, -np.inf], 0)

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_for_clustering)

print(f"Scaled features shape: {scaled_features.shape}")

# --- 1. PCA (Dimensionality Reduction & Visualization) ---
print("\n--- Performing PCA ---")
pca = PCA(n_components=2) # Reduce to 2 dimensions for visualization
pca_result = pca.fit_transform(scaled_features)
print(f"Explained variance ratio (first 2 components): {pca.explained_variance_ratio_}")
print(f"Total explained variance: {pca.explained_variance_ratio_.sum():.4f}")

plt.figure(figsize=(10, 7))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], alpha=0.6)
plt.title('PCA of Manual Features (First 2 Components)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

# --- 2. T-SNE (Dimensionality Reduction & Visualization) ---
print("\n--- Performing T-SNE ---")
# Note: T-SNE can be computationally expensive on large datasets.
# Consider using a subset or PCA first if it's too slow.
n_samples_tsne = min(5000, scaled_features.shape[0]) # Limit samples for performance
indices = np.random.choice(scaled_features.shape[0], n_samples_tsne, replace=False)
scaled_features_subset = scaled_features[indices, :]
# pca_result_subset = pca_result[indices, :] # Can apply t-SNE on PCA results too

tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=42, verbose=1)
tsne_result = tsne.fit_transform(scaled_features_subset) # Use subset for T-SNE

plt.figure(figsize=(10, 7))
sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1], alpha=0.6)
plt.title(f't-SNE of Manual Features ({n_samples_tsne} samples)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True)
plt.show()


# --- 3. K-Means Clustering ---
print("\n--- Performing K-Means Clustering ---")
# Determine optimal K using the Elbow method
inertia = []
k_range = range(1, 11) # Check K from 1 to 10
for k in k_range:
    kmeans_test = KMeans(n_clusters=k, random_state=42, n_init=10) # Suppress warning
    kmeans_test.fit(scaled_features)
    inertia.append(kmeans_test.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia (Within-cluster sum of squares)')
plt.xticks(k_range)
plt.grid(True)
plt.show()

# Choose K based on the elbow plot (e.g., K=4)
optimal_k = 4 # Adjust based on the plot
print(f"Selected K = {optimal_k}")

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(scaled_features)

# Add cluster labels to the original features DataFrame (optional)
manual_features_df['kmeans_cluster'] = kmeans_labels

# Visualize K-Means clusters using PCA results
plt.figure(figsize=(10, 7))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=kmeans_labels, palette='viridis', alpha=0.7, s=50)
plt.title(f'K-Means Clustering (K={optimal_k}) Results on PCA Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

# --- 4. Hierarchical Clustering ---
print("\n--- Performing Hierarchical Clustering ---")
# Generate the linkage matrix using Ward's method (minimizes variance within clusters)
# Use a subset if the dataset is very large, as linkage calculation can be heavy
n_samples_hc = min(2000, scaled_features.shape[0])
indices_hc = np.random.choice(scaled_features.shape[0], n_samples_hc, replace=False)
scaled_features_subset_hc = scaled_features[indices_hc, :]

linked = linkage(scaled_features_subset_hc, method='ward')

# Plot the dendrogram
plt.figure(figsize=(15, 7))
dendrogram(linked,
           orientation='top',
           # labels=manual_features_df.index[indices_hc].tolist(), # Optional: add labels if meaningful/small subset
           distance_sort='descending',
           show_leaf_counts=True,
           truncate_mode='lastp', # Show only the last p merged clusters
           p=30) # Adjust 'p' to control dendrogram complexity
plt.title('Hierarchical Clustering Dendrogram (Ward, Truncated)')
plt.xlabel('Sample Index (or Cluster Size)')
plt.ylabel('Distance (Ward)') 
plt.grid(axis='y')
plt.show()

# Choose the number of clusters based on the dendrogram (e.g., by cutting at a certain height)
num_hierarchical_clusters = 4 # Adjust based on dendrogram inspection
print(f"Selected number of hierarchical clusters = {num_hierarchical_clusters}")

# Apply Agglomerative Clustering
agg_cluster = AgglomerativeClustering(n_clusters=num_hierarchical_clusters, affinity='euclidean', linkage='ward')
hierarchical_labels = agg_cluster.fit_predict(scaled_features) # Fit on all data

# Add cluster labels to the original features DataFrame (optional)
manual_features_df['hierarchical_cluster'] = hierarchical_labels

# Visualize Hierarchical clusters using PCA results
plt.figure(figsize=(10, 7))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=hierarchical_labels, palette='viridis', alpha=0.7, s=50)
plt.title(f'Hierarchical Clustering (n={num_hierarchical_clusters}) Results on PCA Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

### visualizing clusters <a name="visualize-clusters1"></a>

We will visualize the different clusters created from the algorithms and evaluate whether these clusters correspond to specific cognitive patterns, or any other patterns observed between the different clusters.

## Method 2

### Deep feature extraction and Clustering <a name="deep-feature-extraction"></a>

In this section we will use unsupervised deep learning approaches like Autoencoders, Variational Autoencoders (VAE), and LSTM Autoencoders to extract latent features from the raw data or the manually selected features.

### Exploring deep feature clusters

We will again use algorithms such as PCA, t-SNE, k-means and hierarchical clustering to cluster the deep features and visualize the latent space. As done before we will evaluate whether these clusters correspond to specific patterns in the inspections.

## Testing generality of clusters

To test the generality of these cluster we will load the warehouse data. For this dataset the user was asked to find a specific graffiti inside a warehouse full of graffiti. Although the task is very different in nature, the cognitive patterns should be similar, for example the clusters should be capable of identifying when the person is looking at a graffiti and deciding if it is the correct one or not. On the other hand, we might find that the clusters generated don't generalize well to this problem.